In [ ]:
# cd ~/datafiles
# mkdir wind2014
# cd wind2014
# wget http://freo.me/1LpKbGV -O wd2014.zip
# unzip wd2014.zip
# rm wd2014.zip


# dt = parse("datestring")  # returns dateime.datetime
# def date_and_hour(s):
#     dt = parse(s.replace('?',''))
#     hour = dt.hour
#     return (dt.strftime("%Y-%m-%d"), hour)


# Incidents Schema
# |IncidntNum|Category|Descript|DayOfWeek|Date|Time|PdDistrict|Resolution|Address|X|Y|Location|PdId|
# Wind Schema
# |Station_ID|Station_Name|Location_Label|Interval_Minutes|Interval_End_Time|Wind_Velocity_Mtr_Sec|
#     Wind_Direction_Variance_Deg|Wind_Direction_Deg|Ambient_Temperature_Deg_C|Global_Horizontal_Irradiance|

In [44]:
from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

df = sqlc.read.csv('file:///home/oxclo/datafiles/wind2014/*.csv', header='true', inferSchema='true')
# df.show(1)

+----------+--------------------+--------------+----------------+-----------------+---------------------+---------------------------+------------------+-------------------------+----------------------------+
|Station_ID|        Station_Name|Location_Label|Interval_Minutes|Interval_End_Time|Wind_Velocity_Mtr_Sec|Wind_Direction_Variance_Deg|Wind_Direction_Deg|Ambient_Temperature_Deg_C|Global_Horizontal_Irradiance|
+----------+--------------------+--------------+----------------+-----------------+---------------------+---------------------------+------------------+-------------------------+----------------------------+
|      SF15|Warnerville Switc...|   Warnerville|               5| 2014-01-4? 00:05|                0.671|                       9.09|              91.3|                    3.466|                       0.045|
+----------+--------------------+--------------+----------------+-----------------+---------------------+---------------------------+------------------+----------------

In [1]:
from dateutil.parser import parse
from datetime import datetime
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import to_date
from pyspark.sql.functions import hour

from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

df = sqlc.read.csv('file:///home/oxclo/datafiles/wind2014/*.csv', header='true', inferSchema='true')

df = df.withColumn('Interval_End_Time', regexp_replace(df.Interval_End_Time, '(.*)\\? (.*)', '$1 $2'))
df = df.withColumn('zdate', to_date(df.Interval_End_Time)).withColumn('zhour', hour(df.Interval_End_Time))

df = df.filter("Wind_Velocity_Mtr_Sec > 0.0 and Ambient_Temperature_Deg_C > 0.0")\
.dropna(subset=['Wind_Velocity_Mtr_Sec', "Ambient_Temperature_Deg_C", "Interval_End_Time", "Interval_Minutes"])\
.withColumnRenamed('Station_ID', 'station')\
.withColumnRenamed('Wind_Velocity_Mtr_Sec', 'vel')\
.withColumnRenamed('Ambient_Temperature_Deg_C', 'deg')

df = df.select(df.station, df.zdate, df.zhour, df.vel, df.deg).orderBy(df.station, df.zdate, df.zhour)
df = df.groupBy(df.station, df.zdate, df.zhour).avg('vel', 'deg').orderBy(df.station, df.zdate, df.zhour)
df.show(10)


+-------+----------+-----+------------------+------------------+
|station|     zdate|zhour|          avg(vel)|          avg(deg)|
+-------+----------+-----+------------------+------------------+
|   SF04|2014-01-01|    8|          2.236375|            12.085|
|   SF04|2014-01-01|    9|1.5883000000000003|            13.097|
|   SF04|2014-01-01|   10|2.1740833333333334|14.726041666666662|
|   SF04|2014-01-01|   11|2.1182083333333335|16.376458333333332|
|   SF04|2014-01-01|   12| 2.084979166666667|16.627708333333334|
|   SF04|2014-01-01|   13|1.7878958333333337| 17.40083333333334|
|   SF04|2014-01-01|   14|1.5741249999999998| 17.80083333333333|
|   SF04|2014-01-01|   15|1.7833750000000002|16.636458333333334|
|   SF04|2014-01-01|   16|1.6852708333333328|         15.145625|
|   SF04|2014-01-01|   17| 1.765552631578947|14.621842105263156|
+-------+----------+-----+------------------+------------------+
only showing top 10 rows



In [2]:
from scipy.spatial import KDTree
from pyspark.sql.functions import regexp_extract
from pyspark.sql.functions import to_date
from pyspark.sql.functions import year
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import udf
from pyspark.sql.functions import lit
from pyspark.sql.functions import count
from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

sfpds = sqlc.read.csv('file:///home/oxclo/datafiles/incidents/sfpd.csv', header='true', inferSchema='true')
sfpds = sfpds.withColumn('zdate', to_date(from_unixtime(unix_timestamp(sfpds.Date, 'MM/dd/yyy'))))\
.withColumn('zhour', regexp_extract(sfpds.Time, '(\d+):(\d+)', 1))

sfpds = sfpds.filter(year(sfpds.zdate) == 2014)

# NOTE: stations and their locations info from http://freo.me/oxclo-locate
def to_station(y,x):
    station_locations = [\
    [37.7816834,-122.3887657],[37.7469112,-122.4821759],[37.7411022,-120.8041510],[37.4834543,-122.3187302],\
    [37.7576436,-122.3916382],[37.7970013,-122.4140409],[37.7484960,-122.4567461],[37.7288155,-122.4210133],\
    [37.5839487,-121.9499339],[37.7157156,-122.4145311],[37.7329613,-122.5051491],[37.7575891,-122.3923824],\
    [37.7521169,-122.4497687]]
    stations = ["SF18", "SF04", "SF15", "SF17", "SF36", "SF37","SF07", "SF11", "SF12", "SF14", "SF16", "SF19", "SF34"]
    tree = KDTree(station_locations)
    return stations[tree.query([y, x])[1]]

kdtree_udf = udf(to_station)
sfpds = sfpds.withColumn('station', kdtree_udf(sfpds.Y, sfpds.X))

sfpds = sfpds.select(sfpds.station, sfpds.zdate, sfpds.zhour)\
.groupBy(sfpds.station, sfpds.zdate, sfpds.zhour)\
.agg(count('*').alias('incidents'))\
.orderBy('incidents', ascending=False)
sfpds.show(15)

+-------+----------+-----+---------+
|station|     zdate|zhour|incidents|
+-------+----------+-----+---------+
|   SF37|2014-01-01|   00|       38|
|   SF37|2014-06-29|   17|       34|
|   SF37|2014-09-19|   17|       32|
|   SF37|2014-02-24|   17|       30|
|   SF37|2014-02-13|   15|       30|
|   SF37|2014-08-29|   23|       28|
|   SF37|2014-03-06|   16|       27|
|   SF37|2014-02-01|   00|       27|
|   SF37|2014-08-08|   18|       26|
|   SF37|2014-06-29|   18|       26|
|   SF37|2014-08-26|   19|       26|
|   SF37|2014-07-30|   17|       25|
|   SF37|2014-11-25|   18|       25|
|   SF37|2014-03-14|   17|       24|
|   SF37|2014-10-21|   18|       24|
+-------+----------+-----+---------+
only showing top 15 rows



In [3]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.stat import Statistics

cond = [df.station == sfpds.station, df.zdate == sfpds.zdate, df.zhour == sfpds.zhour]
joined = df.join(sfpds, cond)\
.select(df.station, df.zdate, df.zhour, 'avg(deg)', 'avg(vel)', sfpds.incidents)\
.withColumnRenamed('avg(deg)', 't')\
.withColumnRenamed('avg(vel)', 'w')\
.withColumnRenamed('incidents', 'i')\
.orderBy(sfpds.incidents, ascending=False)

joined.show(15)

# remap the data into a vector of [t,w,i]
vecs = joined.rdd.map(lambda x: Vectors.dense(x.t, x.w, x.i))
print(Statistics.corr(vecs))

+-------+----------+-----+------------------+------------------+---+
|station|     zdate|zhour|                 t|                 w|  i|
+-------+----------+-----+------------------+------------------+---+
|   SF37|2014-01-01|    0|11.945208333333333|0.8127083333333336| 38|
|   SF37|2014-03-06|   16|15.738541666666668| 4.125854166666667| 27|
|   SF37|2014-02-01|    0|11.591249999999997| 2.073729166666667| 27|
|   SF37|2014-06-04|   12|18.386666666666667| 4.349277777777776| 22|
|   SF37|2014-06-06|   20|13.848541666666664|3.5619374999999986| 22|
|   SF37|2014-02-01|   23|10.405444444444441| 1.810311111111111| 22|
|   SF37|2014-07-02|   10| 17.80416666666667|         3.2446875| 22|
|   SF37|2014-04-05|   20|         12.803125| 2.430583333333333| 22|
|   SF37|2014-01-06|   20|12.648245614035089|1.4235614035087716| 21|
|   SF37|2014-11-07|   17|17.531166666666667|1.2534666666666667| 21|
|   SF37|2014-04-01|   10|17.093750000000007|1.9050208333333327| 21|
|   SF37|2014-11-01|    1| 13.9001